In [ ]:
import numpy as np
import numba
from matplotlib import pyplot as plt

In [ ]:
from typing import Iterator

def logistic_map_iterator(x0: float, mu: float) -> Iterator[float]:
    xi = x0
    while True:
        xi = mu*(1 - xi)
        yield xi

@numba.njit
def logistic_map(x0: float, mu: float, i: int) -> float:
    xi = x0
    for _ in range(i):
        xi = mu*xi*(1 - xi)
    return xi

In [ ]:
@numba.jit(nopython=True, parallel=True)
def bifurcation_diagram(mu_min: float, mu_max: float, mu_num: int, 
                        xn_num: int, equilibrium_iterations: int = 200) -> tuple[np.ndarray, np.ndarray]:
    """
    return mu, xn

    xn.shape = (mu_num, xm_num)
    """
    mus = np.linspace(mu_min, mu_max, mu_num)
    xns = np.empty((mu_num, xn_num), dtype=np.float64)
    # also, what is code readability? Sounds boring
    for i, mu in enumerate(mus):
        x0s = np.random.random(xn_num)
        for xni in numba.prange(xn_num):
            xns[i, xni] = logistic_map(x0s[xni], mu, equilibrium_iterations)
    return mus, xns

In [ ]:
# Test execution speeds
num_xn = 200
mu, xns = bifurcation_diagram(1, 4, 1000, num_xn) 
# 14.6 s   no jit
#  3.8 s   jit + compile
#  0.1 s   jit

In [ ]:
num_xn = 20
mu, xns = bifurcation_diagram(1, 4, 100, num_xn)

%matplotlib qt5
plt.figure()
plt.scatter(np.column_stack([mu]*num_xn), xns, s=5)
plt.xlabel("$\\mu$")
plt.ylabel("$x_n$")
plt.title("$x_i = \\mu x_{i-1}\\left(1-x_{i-1}\\right)$; "
          "$x_0 \\in \\left[0, 1\\right]$\n"
           f"{num_xn} random $x_0$ per $\\mu$")
plt.show()